# AA 272 Final Project
### Dual-Device GNSS

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import gnss_lib_py as glp

In [ ]:
# Set up file paths
data_path = '2021-07-19-20-49-us-ca-mtv-a' # Location of dataset

pixel4_gnss_path = os.path.join(data_path, 'pixel4/device_gnss.csv')
pixel5_gnss_path = os.path.join(data_path, 'pixel5/device_gnss.csv')

pixel4_gt_path = os.path.join(data_path, 'pixel4/ground_truth.csv')
pixel5_gt_path = os.path.join(data_path, 'pixel5/ground_truth.csv')

In [ ]:
# Load data
derived_data_pixel4 = glp.AndroidDerived2022(pixel4_gnss_path)
derived_data_pixel5 = glp.AndroidDerived2022(pixel5_gnss_path)

gt_data_pixel4 = glp.AndroidGroundTruth2022(pixel4_gt_path)
gt_data_pixel5 = glp.AndroidGroundTruth2022(pixel5_gt_path)

In [ ]:
# Get state estimates
state_estimate_px4 = glp.solve_wls(derived_data_pixel4)
state_estimate_px5 = glp.solve_wls(derived_data_pixel5)

solution_px4 = glp.prepare_kaggle_submission(state_estimate_px4)
solution_px5 = glp.prepare_kaggle_submission(state_estimate_px5)

# Average Solution

In [ ]:
# Compute average positions (indexes manually selected)
avg_lat = (solution_px4[2,:-2] + solution_px5[2,2:]) / 2
avg_lon = (solution_px4[3,:-2] + solution_px5[3,2:]) / 2

# Create average solution NavData
solution_avg = glp.NavData()
solution_avg['lat_avg_deg'] = avg_lat
solution_avg['lon_avg_deg'] = avg_lon

# Format other data for plotting
solution_px4['lat_px4_deg'] = solution_px4['LatitudeDegrees']
solution_px4['lon_px4_deg'] = solution_px4['LongitudeDegrees']
solution_px5['lat_px5_deg'] = solution_px5['LatitudeDegrees']
solution_px5['lon_px5_deg'] = solution_px5['LongitudeDegrees']

In [ ]:
# Plot data
fig = glp.plot_map(solution_px4,solution_px5,solution_avg,gt_data_pixel4)
mapbox = fig['layout']['mapbox']
mapbox['center'] = {'lon': -122.243, 'lat': 37.43608}
mapbox['zoom'] = 18
fig.update_layout(mapbox=mapbox)
fig.show()
fig.write_image("average.png")

# gnss_lib_py EKF with Concatenated Data

This is currently broken.

In [ ]:
df1 = pd.read_csv(pixel4_gnss_path)
df2 = pd.read_csv(pixel5_gnss_path)
combined_dataset = pd.concat([df1, df2], ignore_index=True)
combined_dataset= combined_dataset.sort_values(by='utcTimeMillis')
output_file_path = 'combined_file.csv'
combined_dataset.to_csv(output_file_path, index=False)
combined = glp.AndroidDerived2022(output_file_path)
state_ekf_combined = glp.solve_gnss_ekf(combined)
fig = glp.plot_map(state_ekf_combined)
fig.show()
fig.write_image("concatenated_ekf.png")

# Multi-Device EKF

In [ ]:
# Dictionary containing data for each device
device_dict={0: derived_data_pixel4, 1: derived_data_pixel5}

In [ ]:
def EKF_update(P, mu, R, sat_state, sat_pseudo, dev, dev_num):
    y = sat_pseudo - get_th_pseudo(mu, sat_state, dev)
    H = get_H(mu, sat_state, dev, dev_num)
    K = P @ H.T @ np.linalg.inv(R + H @ P @ H.T)
    mu = mu + K @ y
    P = (np.eye(P.shape[0]) - K @ H) @ P
    return mu, P

def EKF_predict(mu, dt, Q, P, dev_num):
    mu = state_trans(mu, dt, dev_num)
    F = get_F(dt, dev_num)
    P = F @ P @ F.T + Q
    return mu, P

def get_F(dt, dev_num):
    # Get F matrix
    size = dev_num * 4 + 3
    F = np.zeros((size, size))
    # Set diagonal to 1
    np.fill_diagonal(F, 1.0)
    mat_t = np.eye(3) * dt
    for i in range(dev_num):
      row = i * 4
      column = dev_num * 4
      F[row:(row + 3), column:(column + 3)] = np.eye(3) * dt
    return F

def get_H(mu_cur, sat_state, dev, dev_num):
    # Get H matrix
    H = np.zeros((len(sat_state[:, 0]), len(mu_cur)))
    r = get_th_pseudo(mu_cur, sat_state,dev)
    # Build matrix
    for i in range(len(sat_state)):
      j = dev
      H[i, (j * 4):(4 * j + 4)] = np.array([
          (mu_cur[dev * 4] - sat_state[i, 0]) / r[i],
          (mu_cur[dev * 4 + 1] - sat_state[i, 1]) / r[i],
          (mu_cur[dev * 4 + 2] - sat_state[i, 2]) / r[i],
          1])
    return H

def get_th_pseudo(x_est, sat_state, dev):
    # Get theoretical pseudoranges
    x_sv = sat_state[:, 0]
    y_sv = sat_state[:, 1]
    z_sv = sat_state[:, 2]
    b_est = x_est[dev * 4 + 3]
    pseudoranges = np.zeros((len(x_sv), 1))
    for i in range(len(x_sv)):
      pseudoranges[i, 0] = np.linalg.norm(
          x_est[dev * 4:(dev * 4 + 3)] -
          np.array([[x_sv[i], y_sv[i], z_sv[i]]]).T) + b_est
    return pseudoranges

def state_trans(mu_cur, dt, dev_num):
    # Transition model for Kalman filter, constant speed
    mu_next = get_F(dt, dev_num) @ mu_cur
    return mu_next

In [ ]:
# Find overlapping times
def time_array(datafile, time_row, decimals, dev_id):
  times = datafile[time_row]
  times_unique = np.sort(np.unique(np.around(times, decimals=decimals)))
  times = np.zeros((len(times_unique), 2))
  times[:, 0] = times_unique
  times[:, 1] = dev_id
  min = np.min(times[:, 0])
  max = np.max(times[:, 0])
  return times, min, max

begin_time = -np.inf
end_time = np.inf
timearray = np.empty((0, 2))
for i, data in device_dict.items():
  t, min, max = time_array(data, "gps_millis", 4, i)
  if min > begin_time:
    begin_time = min
  if max < end_time:
    end_time = max
  timearray = np.concatenate((timearray, t))
timearray = timearray[(timearray[:, 0] >= begin_time) & (timearray[:, 0] <= end_time)]
timearray = timearray[timearray[:, 0].argsort()]

In [ ]:
# Time delay adjustment
def adjust_time(state):
  omega = np.array([0, 0, 2 * np.pi / 86164.1])
  pos = np.empty((state.shape[0], 3))
  for i in range(state.shape[0]):
    v = np.cross(omega, state[i, :])
    pos[i, :] = state[i, :] - v * 0.079
  return pos

In [ ]:
# Error calculations (based on AA 272 lecture)
def calculate_rmse(gt, estimate):
  indexes = ["x_rx_*_m", "y_rx_*_m", "z_rx_*_m"]
  gt_indexes = [i[0] for i in gt.find_wildcard_indexes(indexes, max_allow=1).values()]
  est_indexes = [i[0] for i in estimate.find_wildcard_indexes(indexes, max_allow=1).values()]
  errors = np.linalg.norm(estimate[est_indexes] - gt[gt_indexes], axis=0)
  rmse = np.sqrt(np.mean(np.array(errors) ** 2))
  return rmse

def plot_errors(gts, estimate):
  gt = glp.NavData()
  for gtx in gts:
    gt.concat(gtx, inplace=True)
  gt.sort(order="gps_millis", inplace=True)
  gt = gt.where("gps_millis", begin_time, "geq").where("gps_millis", end_time, "leq")
  indexes = ["x_rx_*_m", "y_rx_*_m", "z_rx_*_m"]
  gt_indexes = [i[0] for i in gt.find_wildcard_indexes(indexes, max_allow=1).values()]
  est_indexes = [i[0] for i in estimate.find_wildcard_indexes(indexes, max_allow=1).values()]
  errors = np.linalg.norm(estimate[est_indexes] - gt[gt_indexes], axis=0)
  plt.plot(range(len(timearray[:, 0])), errors, '-')
  plt.title('Error over Time')
  plt.xlabel('Time Step')
  plt.ylabel('Error [m]')
  plt.show()
  return

def check_result(gts, estimate):
  gt = glp.NavData()
  for gtx in gts:
    gt.concat(gtx, inplace=True)
  gt.sort(order="gps_millis", inplace=True)
  # If not using overlapping times, comment out next line
  gt = gt.where("gps_millis", begin_time, "geq").where("gps_millis", end_time, "leq")
  rsme_estimate = calculate_rmse(gt, estimate)
  print("The RMSE for the estimate is: ", rsme_estimate)
  fig = glp.plot_map(gt, estimate)
  return fig

In [ ]:
# Initialize state
x0 = np.array([1.0,0,0,0,0,0,0])
first_time = derived_data_pixel4["TimeNanos"][0]
first_sat = derived_data_pixel4.where("TimeNanos", first_time, "eq")
pos_0 = glp.algorithms.snapshot.solve_wls(first_sat)

### EKF (1 Device)

In [ ]:
x = np.array([[pos_0["x_rx_wls_m"],
               pos_0["y_rx_wls_m"],
               pos_0["z_rx_wls_m"],
               pos_0["b_rx_wls_m"],
               0,
               0,
               0]]).T
Q = np.eye(len(x)) * 1.0
P = Q

states1 = []

for frame_time, delta_t, subset in device_dict[1].loop_time("gps_millis", 2):
  pos_sv_m = subset[["x_sv_m", "y_sv_m", "z_sv_m"]].T
  corr_pr_m = subset["corr_pr_m"].reshape(-1, 1)
  # Remove nan indexes
  not_nan_indexes = ~np.isnan(pos_sv_m).any(axis=1)
  pos_sv_m = pos_sv_m[not_nan_indexes]
  corr_pr_m = corr_pr_m[not_nan_indexes]
  R=np.eye(len(corr_pr_m))
  # Prediction step
  x, P = EKF_predict(x, delta_t, Q, P, 1)
  # Update step
  x, P = EKF_update(P, x, R, pos_sv_m, corr_pr_m, 0, 1)
  states1.append([frame_time] + np.squeeze(x).tolist())

states1 = np.array(states1)

In [ ]:
# Process results
state_estimate1 = glp.NavData()
state_estimate1["gps_millis"] = states1[:, 0]
pos = adjust_time(states1[:, 1:4])
state_estimate1["x_rx_ekf_m"] = pos[:, 0]
state_estimate1["y_rx_ekf_m"] = pos[:, 1]
state_estimate1["z_rx_ekf_m"] = pos[:, 2]
lat,lon,alt = glp.ecef_to_geodetic(state_estimate1[[
      "x_rx_ekf_m",
      "y_rx_ekf_m",
      "z_rx_ekf_m"
    ]].reshape(3, -1))
state_estimate1["lat_rx_baseline_ekf_deg"] = lat
state_estimate1["lon_rx_baseline_ekf_deg"] = lon
state_estimate1["alt_rx_baseline_ekf_m"] = alt

In [ ]:
# Plot results
fig = glp.plot_map(gt_data_pixel4, state_estimate1)
mapbox = fig['layout']['mapbox']
mapbox['center'] = {'lon': -122.243, 'lat': 37.43608}
mapbox['zoom'] = 18
fig.update_layout(mapbox=mapbox)
fig.show()
fig.write_image("baseline_ekf.png")

### EKF (2 Device, Unweighted)

In [ ]:
x = np.array([[pos_0["x_rx_wls_m"],
               pos_0["y_rx_wls_m"],
               pos_0["z_rx_wls_m"],
               pos_0["b_rx_wls_m"],
               pos_0["x_rx_wls_m"],
               pos_0["y_rx_wls_m"],
               pos_0["z_rx_wls_m"],
               pos_0["b_rx_wls_m"],
               0,
               0,
               0]]).T
Q = np.eye(len(x)) * 20
P = np.zeros((len(x), len(x), len(device_dict)))
for i in range(len(device_dict)):
  P[:, :, i] = Q

states = []

oldtime = timearray[0, 0]
delta_t_decimals = 2

timearray1 = timearray
for time, device in timearray1:
  delta_t = time - oldtime
  oldtime = time
  subset = device_dict[device].where(
        "gps_millis",
        [time - 10 ** (-delta_t_decimals), time + 10 ** (-delta_t_decimals)],
        condition="between"
      )
  pos_sv_m = subset[["x_sv_m", "y_sv_m", "z_sv_m"]].T
  corr_pr_m = subset["corr_pr_m"].reshape(-1,1)
  # Remove nan indexes
  not_nan_indexes = ~np.isnan(pos_sv_m).any(axis=1)
  pos_sv_m = pos_sv_m[not_nan_indexes]
  corr_pr_m = corr_pr_m[not_nan_indexes]
  R = np.eye(len(corr_pr_m))
  # Predict step
  x, P[:, :, int(0)] = EKF_predict(x,
                                   delta_t,
                                   Q,
                                   P[:, :, int(0)],
                                   len(device_dict))
  # Update step
  x, P[:, :, int(0)] = EKF_update(P[:, :, int(0)],
                                  x,
                                  R,
                                  pos_sv_m,
                                  corr_pr_m,
                                  int(device),
                                  len(device_dict))
  frame_time = time
  states.append([frame_time] + np.squeeze(x).tolist())

states = np.array(states)

In [ ]:
# Process results
state_estimate2 = glp.NavData()
state_estimate2["gps_millis"] = states[:, 0]
pos1 = adjust_time(states[:, 1:4])
pos2 = adjust_time(states[:, 5:8])
state_estimate2["x_rx_ekf_m"] = (pos1[:, 0] + pos2[:, 0]) / 2
state_estimate2["y_rx_ekf_m"] = (pos1[:, 1] + pos2[:, 1]) / 2
state_estimate2["z_rx_ekf_m"] = (pos1[:, 2] + pos2[:, 2]) / 2
lat,lon,alt = glp.ecef_to_geodetic(state_estimate2[[
      "x_rx_ekf_m",
      "y_rx_ekf_m",
      "z_rx_ekf_m"
    ]].reshape(3,-1))
state_estimate2["lat_rx_unweighted_x2_ekf_deg"] = lat
state_estimate2["lon_rx_unweighted_x2_ekf_deg"] = lon
state_estimate2["alt_rx_unweighted_x2_ekf_m"] = alt

In [ ]:
# Plot results
fig = check_result([gt_data_pixel4, gt_data_pixel5], state_estimate2)
mapbox = fig['layout']['mapbox']
mapbox['center'] = {'lon': -122.243, 'lat': 37.43608}
mapbox['zoom'] = 18
fig.update_layout(mapbox=mapbox)
fig.show()
fig.write_image("2_device_unweighted_ekf.png")

### EKF (2 Device, R Weighted by $C/N_{0}$)

In [ ]:
x = np.array([[pos_0["x_rx_wls_m"],
               pos_0["y_rx_wls_m"],
               pos_0["z_rx_wls_m"],
               pos_0["b_rx_wls_m"],
               pos_0["x_rx_wls_m"],
               pos_0["y_rx_wls_m"],
               pos_0["z_rx_wls_m"],
               pos_0["b_rx_wls_m"],
               0,
               0,
               0]]).T
Q = np.eye(len(x)) * 20
P = np.zeros((len(x), len(x), len(device_dict)))
for i in range(len(device_dict)):
  P[:, :, i] = Q

states = []
oldtime = timearray[100,0]
delta_t_decimals = 2

timearray1 = timearray
for time, device in timearray1:
  delta_t = time - oldtime
  oldtime = time
  subset = device_dict[device].where(
      "gps_millis",
      [time - 10 ** (-delta_t_decimals), time + 10 ** (-delta_t_decimals)],
      condition="between")
  pos_sv_m = subset[["x_sv_m", "y_sv_m", "z_sv_m"]].T
  corr_pr_m = subset["corr_pr_m"].reshape(-1,1)
  cn0_vec=subset[["cn0_dbhz"]]
  # Remove nan indexes
  not_nan_indexes = ~np.isnan(pos_sv_m).any(axis=1)
  pos_sv_m = pos_sv_m[not_nan_indexes]
  corr_pr_m = corr_pr_m[not_nan_indexes]
  cn0_vec = cn0_vec[not_nan_indexes]
  cn0_vec = 10 ** (cn0_vec / 10)
  norm = np.linalg.norm(cn0_vec)
  cn0_vec = cn0_vec / norm
  R = np.diag(1 / cn0_vec) * 0.02 # Weighted
  # Predict step
  x, P[:, :, int(0)] = EKF_predict(x,
                                  delta_t,
                                  Q,
                                  P[:, :, int(0)],
                                  len(device_dict))
  # Update step
  x, P[:, :, int(0)] = EKF_update(P[:, :, int(0)],
                                 x,
                                 R,
                                 pos_sv_m,
                                 corr_pr_m,
                                 int(device),
                                 len(device_dict))
  frame_time = time
  states.append([frame_time] + np.squeeze(x).tolist())

states = np.array(states)

In [ ]:
# Process results
state_estimate3 = glp.NavData()
state_estimate3["gps_millis"] = states[:, 0]
pos1 = adjust_time(states[:, 1:4])
pos2 = adjust_time(states[:, 5:8])
state_estimate3["x_rx_ekf_m"] = (pos1[:, 0] + pos2[:, 0]) / 2
state_estimate3["y_rx_ekf_m"] = (pos1[:, 1] + pos2[:, 1]) / 2
state_estimate3["z_rx_ekf_m"] = (pos1[:, 2] + pos2[:, 2]) / 2
lat,lon,alt = glp.ecef_to_geodetic(state_estimate3[[
      "x_rx_ekf_m",
      "y_rx_ekf_m",
      "z_rx_ekf_m"
    ]].reshape(3, -1))
state_estimate3["lat_rx_weighted_x2_ekf_deg"] = lat
state_estimate3["lon_rx_weighted_x2_ekf_deg"] = lon
state_estimate3["alt_rx_weighted_x2_ekf_m"] = alt

In [ ]:
# Plot results
fig = check_result([gt_data_pixel4, gt_data_pixel5], state_estimate3)
mapbox = fig['layout']['mapbox']
mapbox['center'] = {'lon': -122.243, 'lat': 37.43608}
mapbox['zoom'] = 18
fig.update_layout(mapbox=mapbox)
fig.show()
fig.write_image("2_device_weighted_ekf.png")

### EKF (2 Device, Distance Constraint)

In [ ]:
def EKF_update_distance(P, R, x, distance, dev_num, main_dev):
  # Update the distance between the two devices
  # x: state vector, P: covariance matrix
  # dev_num: total number of devices
  # main_dev: device from which the distances are measured
  # distance: array of distances from one device to the other
    y = distance - get_th_distance(x, main_dev, dev_num).T
    H = get_Hdist(x, distance, main_dev)
    K = P @ H.T @ np.linalg.inv(R + H @ P @ H.T)
    x = x + K @ y.T
    P = (np.eye(P.shape[0]) - K @ H) @ P
    return x, P

def get_th_distance(x, main_dev, dev_num):
  # Distance from each device to the main device
  th_distance = np.zeros((dev_num, 1))
  for i in range(dev_num):
    th_distance[i, 0] = np.linalg.norm(x[i * 4:i * 4 + 3] -
                                       x[main_dev * 4:main_dev * 4 + 3])
  return th_distance

def get_Hdist(x, distance, main_dev):
  # x: current state
  # distance: array ordered as the dictionary with distance from main device
  # main_dev: number in the dictionary of the main device
  H = np.zeros((len(distance), len(x)))
  for i in range(len(distance)):
      r = np.linalg.norm(x[i * 4:i * 4 + 3] - x[main_dev * 4:main_dev * 4 + 3])
      if r != 0:
        H[i, (i * 4):(4 * i + 3)]= np.array([
              (x[i * 4, 0] - x[main_dev, 0])/ r,
              (x[i * 4 + 1, 0] - x[4 * main_dev + 1, 0]) / r,
              (x[i * 4 + 2, 0] - x[4 * main_dev + 2, 0]) / r
            ])
  return H

In [ ]:
x = np.array([[pos_0["x_rx_wls_m"],
              pos_0["y_rx_wls_m"],
              pos_0["z_rx_wls_m"],
              pos_0["b_rx_wls_m"],
              pos_0["x_rx_wls_m"],
              pos_0["y_rx_wls_m"],
              pos_0["z_rx_wls_m"],
              pos_0["b_rx_wls_m"],
              0,
              0,
              0]]).T
distance = np.array([0, 0.2])
Q = np.eye(len(x)) * 20
P = np.zeros((len(x), len(x), len(device_dict)))
main_dev = 0
for i in range(len(device_dict)):
  P[:, :, i] = Q

states = []
oldtime = timearray[0, 0]
delta_t_decimals = 2

timearray1 = timearray
for time, device in timearray1:
  delta_t = time - oldtime
  oldtime = time
  subset = device_dict[device].where(
      "gps_millis",
      [time - 10 ** (-delta_t_decimals), time + 10 ** (-delta_t_decimals)],
      condition="between")
  pos_sv_m = subset[["x_sv_m", "y_sv_m", "z_sv_m"]].T
  corr_pr_m = subset["corr_pr_m"].reshape(-1, 1)
  cn0_vec = subset[["cn0_dbhz"]]
  # Remove nan indexes
  not_nan_indexes = ~np.isnan(pos_sv_m).any(axis=1)
  pos_sv_m = pos_sv_m[not_nan_indexes]
  corr_pr_m = corr_pr_m[not_nan_indexes]
  cn0_vec = cn0_vec[not_nan_indexes]
  cn0_vec = 10 ** (cn0_vec / 10)
  norm = np.linalg.norm(cn0_vec)
  cn0_vec = cn0_vec / norm
  R = np.diag(1 / cn0_vec) * 0.22 # Weighted
  # Predict step
  x, P[:, :, int(0)] = EKF_predict(x,
                                   delta_t,
                                   Q,
                                   P[:, :, int(0)],
                                   len(device_dict))
  # Update step
  x, P[:, :, int(0)] = EKF_update(P[:, :, int(0)],
                                  x,
                                  R,
                                  pos_sv_m,
                                  corr_pr_m,
                                  int(device),
                                  len(device_dict))
  # Update again the distance between the two receivers
  R = np.eye(len(device_dict)) * 0.0001
  x, P[:, :, int(0)] = EKF_update_distance(P[:, :, int(0)],
                                           R,
                                           x,
                                           distance,
                                           len(device_dict),
                                           main_dev)
  frame_time = time
  states.append([frame_time] + np.squeeze(x).tolist())

states = np.array(states)

In [ ]:
# Process results
state_estimate4 = glp.NavData()
state_estimate4["gps_millis"] = states[:, 0]
pos1 = adjust_time(states[:, 1:4])
pos2 = adjust_time(states[:, 5:8])
state_estimate4["x_rx_ekf_m"] = (pos1[:, 0] + pos2[:, 0]) / 2
state_estimate4["y_rx_ekf_m"] = (pos1[:, 1] + pos2[:, 1]) / 2
state_estimate4["z_rx_ekf_m"] = (pos1[:, 2] + pos2[:, 2]) / 2
lat,lon,alt = glp.ecef_to_geodetic(state_estimate4[[
      "x_rx_ekf_m",
      "y_rx_ekf_m",
      "z_rx_ekf_m"
    ]].reshape(3,-1))
state_estimate4["lat_rx_const_dist_ekf_deg"] = lat
state_estimate4["lon_rx_const_dist_ekf_deg"] = lon
state_estimate4["alt_rx_const_dist_ekf_m"] = alt

In [ ]:
fig = check_result([gt_data_pixel4, gt_data_pixel5], state_estimate4)
mapbox = fig['layout']['mapbox']
mapbox['center'] = {'lon': -122.243, 'lat': 37.43608}
mapbox['zoom'] = 18
fig.update_layout(mapbox=mapbox)
fig.show()
fig.write_image("distance_constraint_ekf.png")

# Errors

In [ ]:
# Error over time
plot_errors([gt_data_pixel4, gt_data_pixel5], state_estimate4)

In [ ]:
# Google Smartphone Decimeter Challenge Baseline Pixel 4
baseline_px4 = glp.solve_kaggle_baseline(derived_data_pixel4)
x_base_px4, y_base_px4, z_base_px4 = glp.geodetic_to_ecef(baseline_px4[["lat_rx_deg", "lon_rx_deg", "alt_rx_m"]])
baseline_px4["x_rx_baseline_m"] = x_base_px4
baseline_px4["y_rx_baseline_m"] = y_base_px4
baseline_px4["z_rx_baseline_m"] = z_base_px4
calculate_rmse(gt_data_pixel4, baseline_px4)

In [ ]:
# Google Smartphone Decimeter Challenge Baseline Pixel 5
baseline_px5 = glp.solve_kaggle_baseline(derived_data_pixel5)
x_base_px5, y_base_px5, z_base_px5 = glp.geodetic_to_ecef(baseline_px5[["lat_rx_deg", "lon_rx_deg", "alt_rx_m"]])
baseline_px5["x_rx_baseline_m"] = x_base_px5
baseline_px5["y_rx_baseline_m"] = y_base_px5
baseline_px5["z_rx_baseline_m"] = z_base_px5
calculate_rmse(gt_data_pixel5, baseline_px5.where("gps_millis", 1310762976441.0, "geq"))

In [ ]:
# RMSE WLS Pixel 4
calculate_rmse(gt_data_pixel4, state_estimate_px4)

In [ ]:
# RMSE WLS Pixel 5
calculate_rmse(gt_data_pixel5, state_estimate_px5.where("gps_millis", 1310762976441.0, "geq"))

In [ ]:
# RMSE WLS average
state_estimate_avg = glp.NavData()
state_estimate_avg["x_rx_avg_m"] = (state_estimate_px4['x_rx_wls_m'][:-2] + state_estimate_px5['x_rx_wls_m'][2:]) / 2
state_estimate_avg["y_rx_avg_m"] = (state_estimate_px4['y_rx_wls_m'][:-2] + state_estimate_px5['y_rx_wls_m'][2:]) / 2
state_estimate_avg["z_rx_avg_m"] = (state_estimate_px4['z_rx_wls_m'][:-2] + state_estimate_px5['z_rx_wls_m'][2:]) / 2
rmse_4 = calculate_rmse(gt_data_pixel4.where("gps_millis", 1310762977438.0, "geq").where("gps_millis", 1310764871441.0, "leq"), state_estimate_avg)
rmse_5 = calculate_rmse(gt_data_pixel5.where("gps_millis", 1310762977438.0, "geq").where("gps_millis", 1310764871441.0, "leq"), state_estimate_avg)
rmse_wls_avg = (rmse_4 + rmse_5) / 2
print(rmse_wls_avg)

In [ ]:
# RMSE 1-device EKF
calculate_rmse(gt_data_pixel5, state_estimate1.where("gps_millis", 1310762976441.0, "geq"))